# Decarbonization Projcet


# Model Implementation

#### Preparations

Import packages

In [166]:
from gurobipy import *
import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt

Load parameters


In [167]:
parameters = pd.read_csv("data.csv", index_col='source')
parameters = parameters.astype("float64")
parameters

,number,plant_capacity_power_kw,plant_capacity_force_kwh,plant_generate_force_kwh,fixed_cost_power_dollar_kW,fixed_cost_plant_dollar,operating_cost(dollar_kWh),revenues(dollor_kWh),co2(pounds_kWh),capacity_national_kwh,capacity_national_kw,generation_national_kwh,load_factor,capacity_%,generation_%
source,,,,,,,,,,,,,,,
coal,8.0,860112.50,7.534586e+09,1.725150e+09,750.0,6.450844e+08,0.03,0.32,2.26,6.027668e+10,6880900.0,1.380120e+10,0.23,0.08,0.06
natural gas,48.0,244968.75,2.145926e+09,4.730250e+08,600.0,1.469812e+08,0.07,0.32,0.97,1.030045e+11,11758500.0,2.270520e+10,0.22,0.14,0.10
CCGT,23.0,655143.48,5.739057e+09,2.245357e+09,900.0,5.896291e+08,0.07,0.32,0.77,1.319983e+11,15068300.0,5.164320e+10,0.39,0.17,0.23
nuclear,4.0,827450.00,7.248462e+09,6.399750e+09,3100.0,2.565095e+09,0.04,0.32,0.00,2.899385e+10,3309800.0,2.559900e+10,0.88,0.04,0.12
hydro,739.0,37715.83,3.303907e+08,7.711177e+07,3100.0,1.169191e+08,0.01,0.32,0.00,2.441587e+11,27872000.0,5.698560e+10,0.23,0.32,0.26
wind,256.0,58180.08,5.096575e+08,1.478203e+08,3100.0,1.803582e+08,0.01,0.32,0.00,1.304723e+11,14894100.0,3.784200e+10,0.29,0.17,0.17
solar,141.0,45094.33,3.950263e+08,5.525532e+07,4500.0,2.029245e+08,0.01,0.32,0.00,5.569871e+10,6358300.0,7.791000e+09,0.14,0.07,0.04


In [168]:
parameters["capacity_national_kw"]

source
coal            6880900.0
natural gas    11758500.0
CCGT           15068300.0
nuclear         3309800.0
hydro          27872000.0
wind           14894100.0
solar           6358300.0
Name: capacity_national_kw, dtype: float64

In [169]:
# product_material = np.genfromtxt("F:\Learning\DABP\Assignment 1\Pb1_requirements.csv", dtype=float, delimiter=',', encoding='utf-8-sig')
# print("Shape: of product material", product_material.shape)
# resource = np.genfromtxt("F:\Learning\DABP\Assignment 1\Pb1_availability.csv", dtype=float, delimiter=',', encoding='utf-8-sig')
# print("Shape: of resource", resource.shape)
# demand = np.genfromtxt("F:\Learning\DABP\Assignment 1\Pb1_demand.csv", dtype=float, delimiter=',', encoding='utf-8-sig')
# print("Shape: of demand", demand.shape)
# cost = np.genfromtxt("F:\Learning\DABP\Assignment 1\Pb1_holdingcost.csv", dtype=float, delimiter=',', encoding='utf-8-sig')
# print("Shape: of cost", cost.shape)
# unit_profit = np.genfromtxt("F:\Learning\DABP\Assignment 1\Pb1_unitprofit.csv", dtype=float, delimiter=',', encoding='utf-8-sig')
# print("Shape: of profit", unit_profit.shape)
unit_emission = np.array(parameters["co2(pounds_kWh)"])
print("Shape of unit emission:", unit_emission.shape)
unit_cost = np.array(parameters["operating_cost(dollar_kWh)"])
fixed_cost = np.array(parameters["fixed_cost_plant_dollar"])
yearly_demand = np.array([sum(parameters["generation_national_kwh"]) for i in range(10)])
unit_price = 0.32
capacity_force = np.array(parameters["plant_capacity_force_kwh"]) # kwh for one plant for one year
generate_plant = np.array(parameters["plant_generate_force_kwh"])
num_plant_start = np.array(parameters["number"]) # number of plants at year 0
budget_start = 9080*10**6 # million of euros

Shape of unit emission: (7,)


Set up index sets

In [170]:
10**6

1000000

In [171]:
unit_emission

array([2.26, 0.97, 0.77, 0.  , 0.  , 0.  , 0.  ])

In [172]:
years = range(yearly_demand.shape[0])  # 10
sources = range(unit_emission.shape[0])  # 7

#### Set up model

In [173]:
m = Model()

Decision variables

In [174]:
dv_num_plant = m.addVars(sources, years, vtype=GRB.INTEGER, lb=0.0, name="num_plant")  # 7*10
dv_add_plant = m.addVars(sources, years, vtype=GRB.INTEGER, lb=0.0, name="add_plant")  # 7*10
dv_minus_plant = m.addVars(sources, years, vtype=GRB.INTEGER, lb=0.0, name="minus_plant")  # 7*10
dv_yearly_budget = m.addVars(years, name="budget") # 10

Objective function

In [175]:
# Primary Objective: carbon at year 10, set negative in accordance with the MAXIMIZE model sense
m.setObjectiveN(sum(-unit_emission[j] * generate_plant[j] * dv_num_plant[j, 9] for j in sources), index=0, priority=10)
# Objective 2: total profit = revenue - fixed cost - operating cost
sum_revenue = sum(generate_plant[j]*dv_num_plant[j, i]*unit_price for i in years for j in sources)
sum_fixed_cost = sum(fixed_cost[j]*dv_add_plant[j, i] for j in sources for i in years)
sum_operating_cost = sum(unit_cost[j]*generate_plant[j]*dv_num_plant[j, i] for j in sources for i in years)
m.setObjectiveN(sum_revenue + sum_fixed_cost + sum_operating_cost, index=1, priority=5)
m.modelSense = GRB.MAXIMIZE

Constraints

In [176]:
num_plant_start

array([  8.,  48.,  23.,   4., 739., 256., 141.])

In [177]:
# Definition of S = s0 + sum(x)
for i in years:
    for j in sources:
        m.addConstr(dv_num_plant[j, i] == num_plant_start[j] + sum(dv_add_plant[j, k] - dv_minus_plant[j, k] for k in range(0, i)))

# Budget Definition
for i in years:
    if i == 0:
        m.addConstr(dv_yearly_budget[i] <= budget_start)
    else:
        profit_prev_year = yearly_demand[i-1]*unit_price
        m.addConstr(dv_yearly_budget[i] <= profit_prev_year)

# Budget
for i in years:
    # fixed + operating cost
    yearly_cost = sum(fixed_cost[j]*dv_add_plant[j, i] + unit_cost[j]*generate_plant[j]*dv_num_plant[j, i] for j in sources)
    m.addConstr(yearly_cost <= dv_yearly_budget[i])

# Demand vs Capacity
for i in years:
    m.addConstr(sum(capacity_force[j] * dv_num_plant[j, i] for j in sources) >= yearly_demand[i])

# Demand vs Generate
for i in years:
    m.addConstr(sum(generate_plant[j]*dv_num_plant[j, i] for j in sources) >= yearly_demand[i])

# Generate vs Capacity
for i in years:
    m.addConstr(sum(generate_plant[j]*dv_num_plant[j, i] for j in sources) <= sum(capacity_force[j] * dv_num_plant[j, i] for j in sources))

# Can't demolish clean energy
for j in range(3, 7):
    for i in years:
        m.addConstr(dv_minus_plant[j, i] == 0)


#### Solve the model

In [178]:
# Solve
m.optimize()

Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 160 rows, 220 columns and 1110 nonzeros
Model fingerprint: 0xb5be73e9
Variable types: 10 continuous, 210 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 8e+09]
  Objective range  [2e+07, 4e+09]
  Bounds range     [0e+00, 0e+00]
  RHS range        [4e+00, 2e+11]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.

---------------------------------------------------------------------------
Multi-objectives: starting optimization with 2 objectives ... 
---------------------------------------------------------------------------

Multi-objectives: applying initial presolve ...
---------------------------------------------------------------------------

Presolve removed 87 rows and 57 columns
Presolve time: 0.00s
Presolved: 73 rows and 163 columns
-----------------

It takes 0.67 seconds to solve the model.

In [179]:
# get the set of variables
x = m.getVars()

# Ensure status is optimal
assert m.Status == GRB.Status.OPTIMAL

# Query number of multiple objectives, and number of solutions
nSolutions  = m.SolCount
nObjectives = m.NumObj
print('Problem has', nObjectives, 'objectives')
print('Gurobi found', nSolutions, 'solutions')


Problem has 2 objectives
Gurobi found 10 solutions


In [181]:
# For each solution, print value of first three variables, and
# value for each objective function
solutions = []
for s in range(nSolutions):
  # Set which solution we will query from now on
  m.params.SolutionNumber = s

  # Print objective value of this solution in each objective
  print('Solution', s, ':', end='')
  for o in range(nObjectives):
    # Set which objective we will query
    m.params.ObjNumber = o
    # Query the o-th objective value
    print(' ',m.ObjNVal, end='')

  # print first three variables in the solution
  n = min(len(x),70)
  for j in range(n):
    print(x[j].VarName, x[j].Xn, end='')
  print('')

  # query the full vector of the o-th solution
  solutions.append(m.getAttr('Xn',x))


Solution 0 :  0.0  2711024661077.8286num_plant[0,0] 8.0num_plant[0,1] 10.0num_plant[0,2] 25.0num_plant[0,3] 27.0num_plant[0,4] 28.0num_plant[0,5] 28.0num_plant[0,6] 28.0num_plant[0,7] 28.0num_plant[0,8] 28.0num_plant[0,9] 0.0num_plant[1,0] 48.0num_plant[1,1] 48.0num_plant[1,2] 50.0num_plant[1,3] 51.0num_plant[1,4] 51.0num_plant[1,5] 51.0num_plant[1,6] 51.0num_plant[1,7] 51.0num_plant[1,8] 52.0num_plant[1,9] 0.0num_plant[2,0] 23.0num_plant[2,1] 23.0num_plant[2,2] 23.0num_plant[2,3] 23.0num_plant[2,4] 24.0num_plant[2,5] 26.0num_plant[2,6] 29.0num_plant[2,7] 29.0num_plant[2,8] 31.0num_plant[2,9] 0.0num_plant[3,0] 4.0num_plant[3,1] 4.0num_plant[3,2] 24.0num_plant[3,3] 45.0num_plant[3,4] 64.0num_plant[3,5] 81.0num_plant[3,6] 96.0num_plant[3,7] 110.0num_plant[3,8] 122.0num_plant[3,9] 133.0num_plant[4,0] 739.0num_plant[4,1] 740.0num_plant[4,2] 740.0num_plant[4,3] 740.0num_plant[4,4] 740.0num_plant[4,5] 741.0num_plant[4,6] 741.0num_plant[4,7] 741.0num_plant[4,8] 743.0num_plant[4,9] 746.0num_pl